In [2]:
from pdfminer import pdfpage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import nltk
from nltk import sent_tokenize, word_tokenize
import pandas as pd
from io import StringIO
import csv

from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle
import re

In [48]:
def pdf2token(pdf):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams(char_margin=30, line_margin=2, boxes_flow=1)
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    pagenos = set()
    fp = open(pdf, 'rb')
    writer = csv.writer(open(r"C:\Users\33669\PycharmProjects\OCR\trainer\ner_data\training\BSES.csv", 'w', newline=''))

    for pagenumber, page in enumerate(pdfpage.PDFPage.get_pages(fp, check_extractable=True)):
        #print(pagenumber)
        if pagenumber:
            interpreter.process_page(page)
            data = retstr.getvalue()
            sent_text = sent_tokenize(data)
            for sent in sent_text:
                cleansent = clean(sent)
                #print(sent)
                writer.writerow([cleansent.lower()])
                #writer.writerow('\n')

def txttrain(train):
    entpair = []
    eg = []
    ent = []
    print('Runnning text trian')
    with open(train, 'r') as f:
        for line in f:
            text = f.readline()
            junk = ['(',')','"']
            defi = []
            defi = text.lower().split(',')
            defi[1] = defi[1].strip('\n "()')
            for i in defi[1]:
                if i in junk:
                    defi[1] = defi[1].replace(i, '')
            temp = defi[1]
            defi[1] = defi[0]
            defi[0] = temp
            eg.append(defi[0])
            ent.append(defi[1])
        entpair = list(zip(eg,ent))
        #print(entpair)
    return(entpair)
            

    
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    match = []
    i=0
    x = 0
    while x!= -1:
      x = string.find(pattern, i)      
      tup = (x, x+len(pattern))
      match.append(tup)
      i = x+len(pattern)
    match.pop()
    print(match)
    return(match, string)
    '''
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
    '''
        
    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'", '\n']
    text = text.strip('\n ')
    for i in text:        
        if i in filters:
            text = text.replace(i, " ")
            
    return text

def create_data(df, text, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    
    with open(filepath , 'w') as f:
        for sentence in text:
            print(sentence)
            text_ = sentence       
            match_list = []
            for i,y in df:
                print(i)
                a, text_ = matcher(str(sentence), i)
                print(str(a)+str(text_))
                if a:
                    for itr in a:
                        match_list.append((itr[0][0], itr[0][1], y))
            d = mark_sentence(str(sentence), match_list)
            for i in d.keys():
                f.writelines(i + ' ' + d[i] +'\n')
                f.writelines('\n')
            


In [82]:
def main():
   
    filename = 'BSES'
    ## path to save the txt file.
    pdf = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.pdf'
    filepath = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.txt'
    sentoken = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+filename +'.csv'
    final = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+filename +'_final.csv'
    ## creating the file.
    sentences = []
    data = txttrain(filepath)
    pdf2token(pdf)
    with open(sentoken, 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        for row in csv_reader:
            sentences.append(row)            
    create_data(data, sentences, final)
    
if __name__ == '__main__':
    main()
    

[(0, 19), (176, 195)]
(0, 19)
(176, 195)
